# Predykcja i analiza cen samochodów

## Cel Konkursu: opracowanie modelu predykcyjnego umożliwiającego precyzyjne przewidywanie cen pojazdów

## Wstępne przetwarzanie danych

## Wstępne przetwarzanie danych

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

## Wgrywanie zbiorów danych

In [4]:
train_dataset = pd.read_csv('sales_ads_train.csv')
test_dataset = pd.read_csv('sales_ads_test.csv')

### Połączenie obu zbiorów dla zachowania spójności preprocessingu

In [5]:
# Dodanie kolumny do odrónienia train od test
train_dataset["is_train"] = 1
test_dataset["is_train"] = 0

# Połączenie zbiorów
full_dataset = pd.concat([train_dataset, test_dataset], ignore_index=True)

In [6]:
# Wypisanie unikalnych wartości dla każdej kolumny
for column in full_dataset.columns:
    print(f"Kolumna: {column}")
    print(full_dataset[column].unique())
    print("-" * 50)

Kolumna: ID
[     1      2      3 ... 135395 135396 135397]
--------------------------------------------------
Kolumna: Cena
[ 13900.  25900.  35900. ... 677900. 269855.     nan]
--------------------------------------------------
Kolumna: Waluta
['PLN' nan 'EUR']
--------------------------------------------------
Kolumna: Stan
['Used' 'New' nan]
--------------------------------------------------
Kolumna: Marka_pojazdu
['Renault' 'Opel' 'Ford' 'Toyota' 'BMW' 'Audi' 'Volkswagen' 'Hyundai'
 'Kia' 'Nissan' 'Dacia' 'Mazda' 'Volvo' 'Daewoo' 'Fiat' 'Jaguar' 'Škoda'
 'Mitsubishi' 'Peugeot' 'Seat' 'Porsche' 'Honda' 'Jeep' nan 'Citroën'
 'Mercedes-Benz' 'Chevrolet' 'Subaru' 'MINI' 'Alfa Romeo' 'DS Automobiles'
 'Smart' 'Suzuki' 'Buick' 'Ferrari' 'Saab' 'Land Rover' 'Lancia'
 'Cadillac' 'Infiniti' 'Aixam' 'Chrysler' 'SsangYong' 'Lexus' 'Dodge'
 'McLaren' 'Gaz' 'Lada' 'Triumph' 'Lincoln' 'Daihatsu' 'Pontiac' 'Rover'
 'Hummer' 'Tesla' 'Chatenet' 'Wołga' 'Microcar' 'Cupra' 'Maserati' 'RAM'
 'Isuzu' 

In [7]:
record_number = len(full_dataset)
record_number_train = len(train_dataset)
record_number_test = len(test_dataset)

print(f"Liczba wszystkich rekordów: {record_number}")
print(f"Liczba rekordów w grupie treningowej: {record_number_train}")
print(f"Liczba rekordów w grupie testowej: {record_number_test}")

Liczba wszystkich rekordów: 208304
Liczba rekordów w grupie treningowej: 135397
Liczba rekordów w grupie testowej: 72907


In [8]:
# Liczenie brakujących wartości w każdej kolumnie
missing_values = full_dataset.isnull().sum()

# Wypisanie wyników
print(missing_values)

ID                                 0
Cena                           72907
Waluta                          3376
Stan                            3322
Marka_pojazdu                   3351
Model_pojazdu                   3309
Wersja_pojazdu                 72574
Generacja_pojazdu              62833
Rok_produkcji                   3407
Przebieg_km                     4333
Moc_KM                          3962
Pojemnosc_cm3                   5372
Rodzaj_paliwa                   3410
Emisja_CO2                    115861
Naped                          18186
Skrzynia_biegow                 3938
Typ_nadwozia                    3359
Liczba_drzwi                    4902
Kolor                           3463
Kraj_pochodzenia               91902
Pierwszy_wlasciciel           144260
Data_pierwszej_rejestracji    123324
Data_publikacji_oferty          3449
Lokalizacja_oferty              3341
Wyposazenie                     3253
is_train                           0
dtype: int64


## Taking care of missing data

#### ID i Cena nie mają zadnych brakujących wartości, ale pozostałe zmienne mają

#### Stan

In [9]:
# Wypisanie unikalnych wartości

print(f"Kolumna: Stan")
print(full_dataset['Stan'].unique())

# Liczenie brakujących wartości
miss_val_Stan = full_dataset['Stan'].isnull().sum()
used_count = full_dataset['Stan'].value_counts().get("Used", 0)
new_count = full_dataset['Stan'].value_counts().get("New", 0)

print(f"Used: {used_count}")
print(f"New: {new_count}")
print(f"NaN: {miss_val_Stan}")

Kolumna: Stan
['Used' 'New' nan]
Used: 179929
New: 25053
NaN: 3322


##### Założenie: Jeśli samochód przejechał więcej niż 100 km, to zgodnie z definicją jest używany

In [10]:
# Iteracja po indeksach
for index in full_dataset.index:
    if pd.isna(full_dataset.loc[index, 'Stan']):  # Sprawdzenie, czy Stan to NaN
        przebieg = full_dataset.loc[index, 'Przebieg_km']
        # Upewnienie się, że Przebieg_km jest liczbą przed próbą konwersji
        if pd.notna(przebieg):  # Jeśli Przebieg_km nie jest NaN
            przebieg = int(przebieg)  # Konwersja na int
            # Sprawdzenie, czy Przebieg_km jest większy niż 100
            if przebieg > 100:
                full_dataset.loc[index, 'Stan'] = "Used"  # Ustawienie 'Stan' na 1
            

#### Zakodowanie wartości:
#### Stan = 0 to nowy
#### Stan = 1 uzywany

In [11]:
# Zamiana wartości "New" → 0, "Used" → 1, reszta jako NaN
full_dataset['Stan'] = full_dataset['Stan'].apply(lambda x: 0 if x == "New" else (1 if x == "Used" else np.nan))

# Zliczanie wartości
used_count = (full_dataset['Stan'] == 1).sum()
new_count = (full_dataset['Stan'] == 0).sum()
miss_val_Stan = full_dataset['Stan'].isna().sum()

# Wypisanie wyników
print(f"Used: {used_count}")
print(f"New: {new_count}")
print(f"NaN: {miss_val_Stan}")

Used: 182761
New: 25053
NaN: 490


#### Wynik: zmniejszenie liczby NaN w Stan z 3322 do 490

### Marka_pojazdu

In [12]:
marki_count = full_dataset['Marka_pojazdu'].value_counts()
miss_val_marki = full_dataset['Marka_pojazdu'].isnull().sum()
print(marki_count)
print(f"NaN: {miss_val_marki}")

Volkswagen    18151
BMW           16735
Audi          16459
Opel          16068
Ford          15727
              ...  
MAN               1
NSU               1
Santana           1
Scion             1
Talbot            1
Name: Marka_pojazdu, Length: 108, dtype: int64
NaN: 3351


### Model_pojazdu

In [13]:
model_count = full_dataset['Model_pojazdu'].value_counts()
miss_val_model = full_dataset['Model_pojazdu'].isnull().sum()
print(model_count)
print(f"NaN: {miss_val_model}")

Astra        5218
A4           4510
Seria 3      4499
Golf         4099
Passat       3989
             ... 
3500            1
Countach        1
Cressida        1
Interstar       1
GTC4Lusso       1
Name: Model_pojazdu, Length: 1201, dtype: int64
NaN: 3309


#### Załozenie: Jeśli znany jest model, ale brakuje marki to mona uzupełnić markę na podstawie modelu

In [14]:
# Usuwamy wiersze, gdzie Marka oraz Model są NaN
filtered_dataset = full_dataset.dropna(subset=['Marka_pojazdu', 'Model_pojazdu'])

# Tworzymy mapę Model → Marka
model_to_marka = filtered_dataset.set_index('Model_pojazdu')['Marka_pojazdu'].to_dict()

print(model_to_marka)

{'Grand Espace': 'Renault', 'Megane': 'Renault', 'Zafira': 'Opel', 'Focus': 'Ford', 'Avensis': 'Toyota', 'X5': 'BMW', 'A6': 'Audi', 'Scenic': 'Renault', 'A4': 'Audi', 'Golf': 'Volkswagen', 'X5 M': 'BMW', 'A3': 'Audi', 'Tucson': 'Hyundai', 'Soul': 'Kia', 'Passat': 'Volkswagen', 'Qashqai': 'Nissan', 'Duster': 'Dacia', 'Yaris': 'Toyota', '6': 'Mazda', 'XC 40': 'Volvo', 'XC 60': 'Volvo', 'Chairman': 'Daewoo', 'Tipo': 'Fiat', 'Astra': 'Opel', 'XF': 'Jaguar', 'Octavia': 'Škoda', 'Master': 'Renault', 'Seria 1': 'BMW', 'Vectra': 'Opel', 'i40': 'Hyundai', 'Lancer': 'Mitsubishi', '308': 'Peugeot', 'Altea XL': 'Seat', 'X3': 'BMW', 'Ranger': 'Ford', 'Cayenne': 'Porsche', 'Fabia': 'Škoda', 'Sedici': 'Fiat', 'V50': 'Volvo', 'Jazz': 'Honda', 'Grand Cherokee': 'Jeep', 'Doblo': 'Fiat', 'Clio': 'Renault', 'C3': 'Citroën', 'Cherokee': 'Jeep', 'ASX': 'Mitsubishi', 'Sprinter': 'Mercedes-Benz', 'Blazer': 'Chevrolet', 'Ceed': 'Kia', '911': 'Porsche', 'Seria 5': 'BMW', 'Arteon': 'Volkswagen', 'Corsa': 'Opel',

In [15]:
# Uzupełnianie brakujących marek na podstawie mapy
full_dataset['Marka_pojazdu'] = full_dataset.apply(
    lambda row: model_to_marka[row['Model_pojazdu']] if pd.isna(row['Marka_pojazdu']) and row['Model_pojazdu'] in model_to_marka else row['Marka_pojazdu'],
    axis=1
)

In [16]:
miss_val_marki = full_dataset['Marka_pojazdu'].isnull().sum()
print(f"NaN: {miss_val_marki}")

NaN: 80


#### Wynik: zmniejszenie liczby NaN w Marka_pozadu z 3309 do 80

## Inzynieria cech

### Utworzymy zmienną Wiek_pojazdu na podstawie kolumn Rok_produkcji, Data_pierwszej_rejestracji oraz Data_publikacji_oferty pozywając się NaN

#### Wyciąganie roku z Data_pierwszej_rejestracji

In [17]:
# Konwersja kolumny na format datetime
full_dataset['Data_pierwszej_rejestracji'] = pd.to_datetime(full_dataset['Data_pierwszej_rejestracji'], format='%d/%m/%Y', errors='coerce')

# Wyodrębnienie roku rejestracji
full_dataset['Rok_rejestracji'] = full_dataset['Data_pierwszej_rejestracji'].dt.year

# Podgląd wyników
print(full_dataset[['Data_pierwszej_rejestracji', 'Rok_rejestracji']].head())

  Data_pierwszej_rejestracji  Rok_rejestracji
0                        NaT              NaN
1                 2010-06-16           2010.0
2                        NaT              NaN
3                 2007-11-27           2007.0
4                 2013-05-20           2013.0


#### Załozenie: W miejscu brakującego roku produkcji mozemy w przyblizeniu wstawic datę pierwszej rejestracji

In [18]:
# Sprawdzenie liczby brakujących wartości przed uzupełnieniem
print(f"Brakujące wartości w Rok_produkcji przed: {full_dataset['Rok_produkcji'].isna().sum()}")

# Uzupełnianie braków w Rok_produkcji danymi z Rok_rejestracji, jeśli są dostępne
full_dataset['Rok_produkcji'] = full_dataset.apply(
    lambda row: row['Rok_rejestracji'] if pd.isna(row['Rok_produkcji']) and not pd.isna(row['Rok_rejestracji']) else row['Rok_produkcji'], 
    axis=1
)

# Sprawdzenie, ile wartości udało się uzupełnić
print(f"Brakujące wartości w Rok_produkcji po: {full_dataset['Rok_produkcji'].isna().sum()}")


Brakujące wartości w Rok_produkcji przed: 3407
Brakujące wartości w Rok_produkcji po: 2085


#### Załozenie: Dla danego modelu kazdej marki mozna ustalić średni rok produkcji

In [19]:
# Grupowanie po 'Marka_pojazdu' i 'Model_pojazdu' i obliczanie średniego roku produkcji
mean_rok_produkcji = full_dataset.groupby(['Marka_pojazdu', 'Model_pojazdu'])['Rok_produkcji'].mean().reset_index()

# Zaokrąglamy wartości do pełnych lat i konwertujemy na Int64
mean_rok_produkcji['Rok_produkcji'] = mean_rok_produkcji['Rok_produkcji'].round()

In [20]:
# Tworzymy słownik: Marka + Model -> Średni Rok
model_marka_to_mean_rok_produkcji = mean_rok_produkcji.set_index(['Marka_pojazdu', 'Model_pojazdu'])['Rok_produkcji'].to_dict()

In [21]:
# Uzupełnianie brakujących wartości w 'Rok_produkcji' na podstawie średniego roku produkcji dla danej marki i modelu
full_dataset['Rok_produkcji'] = full_dataset.apply(
    lambda row: model_marka_to_mean_rok_produkcji.get((row['Marka_pojazdu'], row['Model_pojazdu'])) 
    if pd.isna(row['Rok_produkcji']) else row['Rok_produkcji'], 
    axis=1
)

In [22]:
miss_val_rok_produkcji = full_dataset['Rok_produkcji'].isnull().sum()
print(f"NaN: {miss_val_rok_produkcji}")

NaN: 53


In [23]:
# Najstarszy rok produkcji
najstarszy_rok = full_dataset['Rok_produkcji'].min()

# Najnowszy rok produkcji
najnowszy_rok = full_dataset['Rok_produkcji'].max()

# Wyświetlenie wyników
print(f"Najstarszy rok produkcji: {najstarszy_rok}")
print(f"Najnowszy rok produkcji: {najnowszy_rok}")

Najstarszy rok produkcji: 1915.0
Najnowszy rok produkcji: 2021.0


#### Wynik: zmniejszenie liczby NaN w Rok_produkcji z 3407 do 53

#### Wyciąganie roku z Data_publikacji_oferty

In [24]:
miss_val_data_publikacji = full_dataset['Data_publikacji_oferty'].isnull().sum()
print(f"NaN: {miss_val_data_publikacji}")

NaN: 3449


In [25]:
# Konwersja kolumny na format datetime
full_dataset['Data_publikacji_oferty'] = pd.to_datetime(full_dataset['Data_publikacji_oferty'], format='%d/%m/%Y', errors='coerce')

# Wyodrębnienie roku rejestracji
full_dataset['Rok_publikacji'] = full_dataset['Data_publikacji_oferty'].dt.year

# Podgląd wyników
print(full_dataset[['Data_publikacji_oferty', 'Rok_publikacji']].head())

  Data_publikacji_oferty  Rok_publikacji
0             2021-04-28          2021.0
1             2021-05-04          2021.0
2             2021-05-03          2021.0
3             2021-05-02          2021.0
4             2021-05-02          2021.0


In [26]:
# Najstarszy rok publikacji
najstarszy_rok_publikacji = full_dataset['Rok_publikacji'].min()

# Najnowszy rok publikacji
najnowszy_rok_publikacji = full_dataset['Rok_publikacji'].max()

# Wyświetlenie wyników
print(f"Najstarszy rok publikacji: {najstarszy_rok_publikacji}")
print(f"Najnowszy rok publikacji: {najnowszy_rok_publikacji}")

Najstarszy rok publikacji: 2021.0
Najnowszy rok publikacji: 2021.0


#### Uwaga: Oferty są z jednego roku

### Utworzymy zmienną Wiek_pojazdu

In [27]:
# Uzupełnianie brakujących wartości w Rok_publikacji na 2021
full_dataset['Rok_publikacji'] = full_dataset['Rok_publikacji'].fillna(2021)

# Obliczenie wieku samochodu: Rok_publikacji - Rok_produkcji
full_dataset['Wiek_pojazdu'] = full_dataset['Rok_publikacji'] - full_dataset['Rok_produkcji']

# Sprawdzenie wyników
print(full_dataset[['Marka_pojazdu', 'Model_pojazdu', 'Rok_produkcji', 'Rok_publikacji', 'Wiek_pojazdu']].head())

  Marka_pojazdu Model_pojazdu  Rok_produkcji  Rok_publikacji  Wiek_pojazdu
0       Renault  Grand Espace         2005.0          2021.0          16.0
1       Renault        Megane         2010.0          2021.0          11.0
2          Opel        Zafira         2015.0          2021.0           6.0
3          Ford         Focus         2007.0          2021.0          14.0
4        Toyota       Avensis         2013.0          2021.0           8.0


### Przebieg_km

In [28]:
miss_val_przebieg = full_dataset['Przebieg_km'].isnull().sum()
print(f"NaN: {miss_val_przebieg}")

NaN: 4333


#### Załozenie: mozna uśrednić przebieg dla danego roku produkcji pojazdu

In [29]:
# Grupowanie po 'Marka_pojazdu', 'Model_pojazdu' i 'Rok_produkcji' i obliczanie średniego przebiegu
mean_przebieg = full_dataset.groupby(['Marka_pojazdu', 'Model_pojazdu', 'Rok_produkcji'])['Przebieg_km'].mean().reset_index()

# Zaokrąglamy wartości do pełnych kilometrów
mean_przebieg['Przebieg_km'] = mean_przebieg['Przebieg_km'].round()

# Tworzymy słownik: Marka + Model + Rok_produkcji -> Średni przebieg
model_marka_rok_to_mean_przebieg = mean_przebieg.set_index(['Marka_pojazdu', 'Model_pojazdu', 'Rok_produkcji'])['Przebieg_km'].to_dict()

# Uzupełnianie brakujących wartości w 'Przebieg_km' na podstawie średniego przebiegu dla danej marki, modelu i roku produkcji
full_dataset['Przebieg_km'] = full_dataset.apply(
    lambda row: model_marka_rok_to_mean_przebieg.get((row['Marka_pojazdu'], row['Model_pojazdu'], row['Rok_produkcji']))
    if pd.isna(row['Przebieg_km']) else row['Przebieg_km'], 
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Marka_pojazdu', 'Model_pojazdu', 'Rok_produkcji', 'Przebieg_km']].head())

  Marka_pojazdu Model_pojazdu  Rok_produkcji  Przebieg_km
0       Renault  Grand Espace         2005.0     213000.0
1       Renault        Megane         2010.0     117089.0
2          Opel        Zafira         2015.0     115600.0
3          Ford         Focus         2007.0     218000.0
4        Toyota       Avensis         2013.0     174645.0


In [30]:
miss_val_przebieg = full_dataset['Przebieg_km'].isnull().sum()
print(f"NaN: {miss_val_przebieg}")

NaN: 114


#### Wynik: zmniejszenie liczby NaN w Przebieg z 4333 do 114

## Ustalenie waluty

### Waluta

In [31]:
waluta_count = full_dataset['Waluta'].value_counts()
miss_waluta = full_dataset['Waluta'].isnull().sum()
print(waluta_count)
print(f"NaN: {miss_waluta}")

PLN    204664
EUR       264
Name: Waluta, dtype: int64
NaN: 3376


#### Zalozenie: dla kazdego modelu i roku produkcji ustalimy srednia cene i sprawdzimy, czy brakujące waluty to EUR czy PLN

#### Kurs średni Euro za rok 2021 ze strony NBP: 4,5674

In [32]:
# Kurs wymiany z EUR na PLN
kurs_wymiany = 4.5674

In [33]:
# Przeliczenie ceny, jeżeli waluta to EUR
full_dataset['Cena'] = full_dataset.apply(
    lambda row: row['Cena'] * kurs_wymiany if row['Waluta'] == 'EUR' else row['Cena'],
    axis=1
)

In [34]:
# Konwertowanie kolumny Cena na numeryczną (w razie potrzeby)
full_dataset['Cena'] = pd.to_numeric(full_dataset['Cena'], errors='coerce')

# Grupowanie po 'Model_pojazdu' i 'Rok_produkcji' i obliczanie średnich cen w PLN i EUR
mean_prices = full_dataset.groupby(['Model_pojazdu', 'Rok_produkcji', 'Waluta'])['Cena'].mean().reset_index()

# Ustawienie średnich cen w PLN i EUR w osobnych kolumnach
mean_prices_pln = mean_prices[mean_prices['Waluta'] == 'PLN'][['Model_pojazdu', 'Rok_produkcji', 'Cena']].rename(columns={'Cena': 'Mean_Price_PLN'})
mean_prices_eur = mean_prices[mean_prices['Waluta'] == 'EUR'][['Model_pojazdu', 'Rok_produkcji', 'Cena']].rename(columns={'Cena': 'Mean_Price_EUR'})

# Łączenie średnich cen w PLN i EUR z głównym datasetem
full_dataset = full_dataset.merge(mean_prices_pln, on=['Model_pojazdu', 'Rok_produkcji'], how='left')
full_dataset = full_dataset.merge(mean_prices_eur, on=['Model_pojazdu', 'Rok_produkcji'], how='left')

# Uzupełnianie brakujących wartości w 'Waluta' na podstawie średnich cen i porównania z 'Cena'
full_dataset['Waluta'] = full_dataset.apply(
    lambda row: 'PLN' if pd.isna(row['Waluta']) and 
               (abs(row['Cena'] - row['Mean_Price_PLN']) < abs(row['Cena'] - row['Mean_Price_EUR'])) 
               else 
               ('EUR' if pd.isna(row['Waluta']) else row['Waluta']),
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Model_pojazdu', 'Rok_produkcji', 'Cena', 'Waluta', 'Mean_Price_PLN', 'Mean_Price_EUR']].head())


  Model_pojazdu  Rok_produkcji     Cena Waluta  Mean_Price_PLN  Mean_Price_EUR
0  Grand Espace         2005.0  13900.0    PLN    10299.750000             NaN
1        Megane         2010.0  25900.0    PLN    19737.512821             NaN
2        Zafira         2015.0  35900.0    PLN    42233.088235             NaN
3         Focus         2007.0   5999.0    PLN    10209.651685             NaN
4       Avensis         2013.0  44800.0    PLN    39594.416667             NaN


In [35]:
miss_waluta = full_dataset['Waluta'].isnull().sum()
print(f"NaN: {miss_waluta}")

NaN: 0


#### Wynik: Zmniejszenie liczby NaN w Waluta z 3376 do 0

#### Załozenie: Moc silnika zalezy od rodzaju paliwa i danego modelu

In [36]:
miss_moc = full_dataset['Moc_KM'].isnull().sum()
print(f"NaN: {miss_moc}")

NaN: 3962


In [37]:
# Grupowanie po 'Model_pojazdu' i 'Rodzaj_paliwa' i obliczanie średniej mocy
mean_moc = full_dataset.groupby(['Model_pojazdu', 'Rodzaj_paliwa'])['Moc_KM'].mean().reset_index()

# Zaokrąglamy wartości do pełnych KM
mean_moc['Moc_KM'] = mean_moc['Moc_KM'].round()

# Tworzymy słownik: Model + Rodzaj_paliwa -> Średnia moc
model_paliwo_to_mean_moc = mean_moc.set_index(['Model_pojazdu', 'Rodzaj_paliwa'])['Moc_KM'].to_dict()

# Uzupełnianie brakujących wartości w 'Moc' na podstawie średniego przebiegu dla danej marki, modelu i roku produkcji
full_dataset['Moc_KM'] = full_dataset.apply(
    lambda row: model_paliwo_to_mean_moc.get((row['Model_pojazdu'], row['Rodzaj_paliwa']))
    if pd.isna(row['Moc_KM']) else row['Moc_KM'], 
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Marka_pojazdu', 'Model_pojazdu', 'Moc_KM', 'Rodzaj_paliwa']].head())

  Marka_pojazdu Model_pojazdu  Moc_KM Rodzaj_paliwa
0       Renault  Grand Espace   170.0      Gasoline
1       Renault        Megane   110.0      Gasoline
2          Opel        Zafira   136.0           NaN
3          Ford         Focus    90.0        Diesel
4        Toyota       Avensis   142.0      Gasoline


In [38]:
miss_moc = full_dataset['Moc_KM'].isnull().sum()
print(f"NaN: {miss_moc}")

NaN: 194


#### Wynik: zmniejszenie NaN w Moc silnika z 3962 do 194

#### Załozenie: Pojemność silnika zalezy od mocy silnika i danego modelu

In [39]:
miss_pojemnosc = full_dataset['Pojemnosc_cm3'].isnull().sum()
print(f"NaN: {miss_pojemnosc}")

# Grupowanie po 'Model_pojazdu' i 'Moc_KM' i obliczanie średniej mocy
mean_moc = full_dataset.groupby(['Model_pojazdu', 'Moc_KM'])['Pojemnosc_cm3'].mean().reset_index()

# Zaokrąglamy wartości do pełnych KM
mean_moc['Pojemnosc_cm3'] = mean_moc['Pojemnosc_cm3'].round()

# Tworzymy słownik: Model + Moc_KM -> Średnia pojemnosc
model_paliwo_to_mean_moc = mean_moc.set_index(['Model_pojazdu', 'Moc_KM'])['Pojemnosc_cm3'].to_dict()

# Uzupełnianie brakujących wartości w Pojemnosc na podstawie średniego przebiegu dla modelu i mocy silnika
full_dataset['Pojemnosc_cm3'] = full_dataset.apply(
    lambda row: model_paliwo_to_mean_moc.get((row['Model_pojazdu'], row['Moc_KM']))
    if pd.isna(row['Pojemnosc_cm3']) else row['Pojemnosc_cm3'], 
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Marka_pojazdu', 'Model_pojazdu', 'Pojemnosc_cm3', 'Moc_KM']].head())

miss_pojemnosc = full_dataset['Pojemnosc_cm3'].isnull().sum()
print(f"NaN: {miss_pojemnosc}")

NaN: 5372
  Marka_pojazdu Model_pojazdu  Pojemnosc_cm3  Moc_KM
0       Renault  Grand Espace         1998.0   170.0
1       Renault        Megane         1598.0   110.0
2          Opel        Zafira         1598.0   136.0
3          Ford         Focus         1560.0    90.0
4        Toyota       Avensis         1798.0   142.0
NaN: 1292


#### Wynik: zmniejszenie NaN w Pojemność silnika z 5372 do 1292

#### Załozenie: Rodzaj_paliwa zalezy od modelu i mocy

In [40]:
paliwo_count = full_dataset['Rodzaj_paliwa'].value_counts()
miss_paliwo = full_dataset['Rodzaj_paliwa'].isnull().sum()
print(paliwo_count)
print(f"NaN: {miss_paliwo}")

Gasoline          97773
Diesel            90851
Gasoline + LPG     8882
Hybrid             5792
Electric           1523
Gasoline + CNG       69
Hydrogen              2
Ethanol               2
Name: Rodzaj_paliwa, dtype: int64
NaN: 3410


In [41]:
# Sprawdzenie brakujących danych w kolumnie 'Rodzaj_paliwa'
miss_paliwo = full_dataset['Rodzaj_paliwa'].isnull().sum()
print(f"NaN: {miss_paliwo}")

# Grupowanie po 'Model_pojazdu' i 'Moc_KM' i obliczanie najczęstszego rodzaju paliwa
mode_paliwo = full_dataset.groupby(['Model_pojazdu', 'Moc_KM'])['Rodzaj_paliwa'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()

# Tworzymy słownik: Model + Moc_KM -> Najczęstszy rodzaj paliwa
model_moc_to_paliwo = mode_paliwo.set_index(['Model_pojazdu', 'Moc_KM'])['Rodzaj_paliwa'].to_dict()

# Uzupełnianie brakujących wartości w 'Rodzaj_paliwa' na podstawie najczęstszego paliwa dla modelu i mocy
full_dataset['Rodzaj_paliwa'] = full_dataset.apply(
    lambda row: model_moc_to_paliwo.get((row['Model_pojazdu'], row['Moc_KM']))
    if pd.isna(row['Rodzaj_paliwa']) else row['Rodzaj_paliwa'], 
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Marka_pojazdu', 'Model_pojazdu', 'Rodzaj_paliwa', 'Moc_KM']].head())

# Sprawdzanie brakujących wartości po uzupełnieniu
miss_paliwo = full_dataset['Rodzaj_paliwa'].isnull().sum()
print(f"NaN: {miss_paliwo}")


NaN: 3410
  Marka_pojazdu Model_pojazdu Rodzaj_paliwa  Moc_KM
0       Renault  Grand Espace      Gasoline   170.0
1       Renault        Megane      Gasoline   110.0
2          Opel        Zafira        Diesel   136.0
3          Ford         Focus        Diesel    90.0
4        Toyota       Avensis      Gasoline   142.0
NaN: 250


#### Wynik: zmniejszenie NaN w Rodzaj_paliwa z 3410 do 250

#### Załozenie: Emisja_CO2 zalezy od Rodzaju paliwa, Pojemności silnika

In [42]:
# Liczba brakujących wartości w Emisja_CO2
miss_emisja = full_dataset['Emisja_CO2'].isnull().sum()
print(f"NaN przed uzupełnieniem: {miss_emisja}")

# Grupowanie po 'Rodzaj_paliwa' i 'Moc_KM' i obliczanie średniej emisji CO2
mean_emisja = full_dataset.groupby(['Rodzaj_paliwa', 'Moc_KM'])['Emisja_CO2'].mean().reset_index()

# Zaokrąglamy wartości emisji CO2
mean_emisja['Emisja_CO2'] = mean_emisja['Emisja_CO2'].round()

# Tworzymy słownik: (Rodzaj_paliwa, Moc_KM) -> Średnia emisja CO2
paliwo_moc_to_mean_emisja = mean_emisja.set_index(['Rodzaj_paliwa', 'Moc_KM'])['Emisja_CO2'].to_dict()

# Uzupełnianie brakujących wartości w Emisja_CO2 na podstawie średniej emisji dla danego paliwa i mocy silnika
full_dataset['Emisja_CO2'] = full_dataset.apply(
    lambda row: paliwo_moc_to_mean_emisja.get((row['Rodzaj_paliwa'], row['Moc_KM']))
    if pd.isna(row['Emisja_CO2']) else row['Emisja_CO2'], 
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Marka_pojazdu', 'Model_pojazdu', 'Emisja_CO2', 'Moc_KM']].head())

# Liczba brakujących wartości po uzupełnieniu
miss_emisja = full_dataset['Emisja_CO2'].isnull().sum()
print(f"NaN po uzupełnieniu: {miss_emisja}")

NaN przed uzupełnieniem: 115861
  Marka_pojazdu Model_pojazdu  Emisja_CO2  Moc_KM
0       Renault  Grand Espace       232.0   170.0
1       Renault        Megane       159.0   110.0
2          Opel        Zafira       145.0   136.0
3          Ford         Focus       118.0    90.0
4        Toyota       Avensis       152.0   142.0
NaN po uzupełnieniu: 2655


#### Wynik: zmniejszenie NaN w Emisja_CO2 z 115861 do 2665

#### Załozenie: Typ_nadwozia zalezy od modelu

In [43]:
miss_nadwozie = full_dataset['Typ_nadwozia'].isnull().sum()
print(f"NaN: {miss_nadwozie}")

# Grupowanie po 'Model_pojazdu' obliczanie najczęstszego typu nadwozia
mode_paliwo = full_dataset.groupby(['Model_pojazdu'])['Typ_nadwozia'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()

# Tworzymy słownik
model_moc_to_paliwo = mode_paliwo.set_index(['Model_pojazdu'])['Typ_nadwozia'].to_dict()

# Uzupełnianie brakujących wartości
full_dataset['Typ_nadwozia'] = full_dataset.apply(
    lambda row: model_moc_to_paliwo.get((row['Model_pojazdu']))
    if pd.isna(row['Typ_nadwozia']) else row['Typ_nadwozia'], 
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Model_pojazdu', 'Typ_nadwozia']].head())

# Sprawdzanie brakujących wartości po uzupełnieniu
miss_nadwozie = full_dataset['Typ_nadwozia'].isnull().sum()
print(f"NaN: {miss_nadwozie}")


NaN: 3359
  Model_pojazdu   Typ_nadwozia
0  Grand Espace        minivan
1        Megane  station_wagon
2        Zafira        minivan
3         Focus        compact
4       Avensis  station_wagon
NaN: 67


#### Wynik: zmniejszenie NaN w Typ_nadwozia z 3359 do 67

#### Załozenie: Liczba_drzwi zalezy od Typu nadwozia i Modelu

In [44]:
# Sprawdzenie brakujących danych
miss_drzwi = full_dataset['Liczba_drzwi'].isnull().sum()
print(f"NaN: {miss_drzwi}")

# Grupowanie
mode_paliwo = full_dataset.groupby(['Model_pojazdu', 'Typ_nadwozia'])['Liczba_drzwi'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()

# Tworzymy słownik
model_moc_to_paliwo = mode_paliwo.set_index(['Model_pojazdu', 'Typ_nadwozia'])['Liczba_drzwi'].to_dict()

# Uzupełnianie brakujących wartości
full_dataset['Liczba_drzwi'] = full_dataset.apply(
    lambda row: model_moc_to_paliwo.get((row['Model_pojazdu'], row['Typ_nadwozia']))
    if pd.isna(row['Liczba_drzwi']) else row['Liczba_drzwi'], 
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Model_pojazdu', 'Typ_nadwozia', 'Liczba_drzwi']].head())

# Sprawdzenie brakujących danych
miss_drzwi = full_dataset['Liczba_drzwi'].isnull().sum()
print(f"NaN: {miss_drzwi}")


NaN: 4902
  Model_pojazdu   Typ_nadwozia  Liczba_drzwi
0  Grand Espace        minivan           5.0
1        Megane  station_wagon           5.0
2        Zafira        minivan           5.0
3         Focus        compact           5.0
4       Avensis  station_wagon           4.0
NaN: 147


#### Wynik: zmniejszenie NaN w Typ_nadwozia z 4902 do 147

#### Zalozenie: Naped zalezy od modelu

In [45]:
miss_nadwozie = full_dataset['Naped'].isnull().sum()
print(f"NaN: {miss_nadwozie}")

# Grupowanie po 'Model_pojazdu' obliczanie najczęstszego typu nadwozia
mode_paliwo = full_dataset.groupby(['Model_pojazdu'])['Naped'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()

# Tworzymy słownik
model_moc_to_paliwo = mode_paliwo.set_index(['Model_pojazdu'])['Naped'].to_dict()

# Uzupełnianie brakujących wartości
full_dataset['Naped'] = full_dataset.apply(
    lambda row: model_moc_to_paliwo.get((row['Model_pojazdu']))
    if pd.isna(row['Naped']) else row['Naped'], 
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Model_pojazdu', 'Naped']].head())

# Sprawdzanie brakujących wartości po uzupełnieniu
miss_nadwozie = full_dataset['Naped'].isnull().sum()
print(f"NaN: {miss_nadwozie}")



NaN: 18186
  Model_pojazdu         Naped
0  Grand Espace  Front wheels
1        Megane  Front wheels
2        Zafira  Front wheels
3         Focus  Front wheels
4       Avensis  Front wheels
NaN: 411


In [46]:
miss_nadwozie = full_dataset['Skrzynia_biegow'].isnull().sum()
print(f"NaN: {miss_nadwozie}")

# Grupowanie po 'Model_pojazdu' obliczanie najczęstszego typu nadwozia
mode_paliwo = full_dataset.groupby(['Model_pojazdu'])['Skrzynia_biegow'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()

# Tworzymy słownik
model_moc_to_paliwo = mode_paliwo.set_index(['Model_pojazdu'])['Skrzynia_biegow'].to_dict()

# Uzupełnianie brakujących wartości
full_dataset['Skrzynia_biegow'] = full_dataset.apply(
    lambda row: model_moc_to_paliwo.get((row['Model_pojazdu']))
    if pd.isna(row['Skrzynia_biegow']) else row['Skrzynia_biegow'], 
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Model_pojazdu', 'Skrzynia_biegow']].head())

# Sprawdzanie brakujących wartości po uzupełnieniu
miss_nadwozie = full_dataset['Skrzynia_biegow'].isnull().sum()
print(f"NaN: {miss_nadwozie}")



NaN: 3938
  Model_pojazdu Skrzynia_biegow
0  Grand Espace          Manual
1        Megane          Manual
2        Zafira          Manual
3         Focus          Manual
4       Avensis          Manual
NaN: 90


In [47]:
miss_kolor = full_dataset['Kolor'].isnull().sum()
print(f"NaN: {miss_kolor}")

# Uzupełnianie brakujących wartości
full_dataset['Kolor'] = full_dataset.apply(
    lambda row: "other" if pd.isna(row['Kolor']) else row['Kolor'], 
    axis=1
)

# Sprawdzenie wyników
print(full_dataset[['Model_pojazdu', 'Kolor']].head())

# Sprawdzanie brakujących wartości po uzupełnieniu
miss_kolor = full_dataset['Kolor'].isnull().sum()
print(f"NaN: {miss_kolor}")

NaN: 3463
  Model_pojazdu   Kolor
0  Grand Espace    blue
1        Megane  silver
2        Zafira   white
3         Focus    blue
4       Avensis   other
NaN: 0


## Zmiana typu danych

### Odrzucanie kolumn wyłączonych z modelu

In [48]:
for column in full_dataset:
    print(column)

ID
Cena
Waluta
Stan
Marka_pojazdu
Model_pojazdu
Wersja_pojazdu
Generacja_pojazdu
Rok_produkcji
Przebieg_km
Moc_KM
Pojemnosc_cm3
Rodzaj_paliwa
Emisja_CO2
Naped
Skrzynia_biegow
Typ_nadwozia
Liczba_drzwi
Kolor
Kraj_pochodzenia
Pierwszy_wlasciciel
Data_pierwszej_rejestracji
Data_publikacji_oferty
Lokalizacja_oferty
Wyposazenie
is_train
Rok_rejestracji
Rok_publikacji
Wiek_pojazdu
Mean_Price_PLN
Mean_Price_EUR


# # #

In [49]:
kolumny_do_usuniecia = ['Wersja_pojazdu', 'Generacja_pojazdu', 'Kraj_pochodzenia', 'Pierwszy_wlasciciel', 'Data_pierwszej_rejestracji', 'Data_publikacji_oferty', 'Lokalizacja_oferty', 'Rok_rejestracji', 'Wyposazenie', 'Mean_Price_PLN', 'Mean_Price_EUR']  # Wpisz nazwy kolumn do usunięcia
full_dataset_drop = full_dataset.drop(columns=kolumny_do_usuniecia)

In [50]:
# Sprawdzanie typu danych w każdej kolumnie
print(full_dataset_drop.dtypes)

ID                   int64
Cena               float64
Waluta              object
Stan               float64
Marka_pojazdu       object
Model_pojazdu       object
Rok_produkcji      float64
Przebieg_km        float64
Moc_KM             float64
Pojemnosc_cm3      float64
Rodzaj_paliwa       object
Emisja_CO2         float64
Naped               object
Skrzynia_biegow     object
Typ_nadwozia        object
Liczba_drzwi       float64
Kolor               object
is_train             int64
Rok_publikacji     float64
Wiek_pojazdu       float64
dtype: object


In [51]:
# Liczenie brakujących wartości w każdej kolumnie
missing_values = full_dataset_drop.isnull().sum()

# Wypisanie wyników
print(missing_values)

ID                     0
Cena               72907
Waluta                 0
Stan                 490
Marka_pojazdu         80
Model_pojazdu       3309
Rok_produkcji         53
Przebieg_km          114
Moc_KM               194
Pojemnosc_cm3       1292
Rodzaj_paliwa        250
Emisja_CO2          2655
Naped                411
Skrzynia_biegow       90
Typ_nadwozia          67
Liczba_drzwi         147
Kolor                  0
is_train               0
Rok_publikacji         0
Wiek_pojazdu          53
dtype: int64


### Usunięcie rekordów z NaN pozostałych pomimo imputowania

In [ ]:
# Sprawdzanie liczby wierszy przed usunięciem
before_drop = len(full_dataset_drop)

# Usunięcie wszystkich wierszy, które zawierają przynajmniej jeden NaN za wyjątkiem Cena (wszystkie ceny są puste test_dataset)
full_dataset_drop = full_dataset_drop.dropna(subset=[col for col in full_dataset_drop.columns if col != 'Cena'])

# Sprawdzanie liczby wierszy po usunięciu
after_drop = len(full_dataset_drop)

# Obliczanie liczby usuniętych wierszy
removed_rows = before_drop - after_drop

# Wyświetlanie wyniku
print(f"Liczba wierszy przed usunięciem: {before_drop}")
print(f"Liczba usuniętych wierszy: {removed_rows}")
print(f"Liczba wierszy po usunięciu: {after_drop}")

In [53]:
from sklearn.impute import SimpleImputer

# Sprawdzanie liczby wierszy przed wypełnieniem NaN
before_imputation = len(full_dataset_drop)

# Kolumny numeryczne (wykluczając 'Cena')
numeric_columns = [col for col in full_dataset_drop.columns if full_dataset_drop[col].dtype in ['float64', 'int64'] and col != 'Cena']

# Kolumny kategoryczne
categorical_columns = [col for col in full_dataset_drop.columns if full_dataset_drop[col].dtype == 'object' and col != 'Cena']

# Tworzenie SimpleImputer dla danych numerycznych (średnia)
numeric_imputer = SimpleImputer(strategy='mean')

# Tworzenie SimpleImputer dla danych kategorycznych (najczęstsza wartość)
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Wypełnianie brakujących wartości w kolumnach numerycznych
full_dataset_drop[numeric_columns] = numeric_imputer.fit_transform(full_dataset_drop[numeric_columns])

# Wypełnianie brakujących wartości w kolumnach kategorycznych
full_dataset_drop[categorical_columns] = categorical_imputer.fit_transform(full_dataset_drop[categorical_columns])

# Sprawdzanie liczby wierszy po wypełnieniu NaN
after_imputation = len(full_dataset_drop)

# Obliczanie liczby wierszy, które zostały zmodyfikowane
removed_rows = before_imputation - after_imputation

# Wyświetlanie wyniku
print(f"Liczba wierszy przed wypełnieniem NaN: {before_imputation}")
print(f"Liczba wierszy po wypełnieniu NaN: {after_imputation}")


Liczba wierszy przed wypełnieniem NaN: 208304
Liczba wierszy po wypełnieniu NaN: 208304


### Kodowanie danych kategorialnych

In [54]:
categorical_features = full_dataset_drop.select_dtypes(include=['object']).columns
print("Kolumny kategorialne:", categorical_features.tolist())

Kolumny kategorialne: ['Waluta', 'Marka_pojazdu', 'Model_pojazdu', 'Rodzaj_paliwa', 'Naped', 'Skrzynia_biegow', 'Typ_nadwozia', 'Kolor']


#### Nie wiem dlaczego nie działą tu OneHotEncoder, zamiast tego get_dummies

In [55]:
print(full_dataset_drop.shape)

# Wykorzystanie pandas.get_dummies() do zakodowania zmiennych kategorialnych
full_dataset_transformed = pd.get_dummies(full_dataset_drop, columns=categorical_features, drop_first=True)

# Wynik
print(full_dataset_transformed.shape)

(208304, 20)
(208304, 1353)


In [56]:
for column in full_dataset_transformed:
    print(column)

ID
Cena
Stan
Rok_produkcji
Przebieg_km
Moc_KM
Pojemnosc_cm3
Emisja_CO2
Liczba_drzwi
is_train
Rok_publikacji
Wiek_pojazdu
Waluta_PLN
Marka_pojazdu_Acura
Marka_pojazdu_Aixam
Marka_pojazdu_Alfa Romeo
Marka_pojazdu_Alpine
Marka_pojazdu_Aston Martin
Marka_pojazdu_Audi
Marka_pojazdu_Austin
Marka_pojazdu_Autobianchi
Marka_pojazdu_BMW
Marka_pojazdu_Baic
Marka_pojazdu_Bentley
Marka_pojazdu_Buick
Marka_pojazdu_Cadillac
Marka_pojazdu_Casalini
Marka_pojazdu_Chatenet
Marka_pojazdu_Chevrolet
Marka_pojazdu_Chrysler
Marka_pojazdu_Citroën
Marka_pojazdu_Cupra
Marka_pojazdu_DFSK
Marka_pojazdu_DKW
Marka_pojazdu_DS Automobiles
Marka_pojazdu_Dacia
Marka_pojazdu_Daewoo
Marka_pojazdu_Daihatsu
Marka_pojazdu_Dodge
Marka_pojazdu_FAW
Marka_pojazdu_Ferrari
Marka_pojazdu_Fiat
Marka_pojazdu_Ford
Marka_pojazdu_GMC
Marka_pojazdu_Gaz
Marka_pojazdu_Grecav
Marka_pojazdu_Honda
Marka_pojazdu_Hummer
Marka_pojazdu_Hyundai
Marka_pojazdu_Infiniti
Marka_pojazdu_Inny
Marka_pojazdu_Isuzu
Marka_pojazdu_Iveco
Marka_pojazdu_Jaguar
M

### Ponowne podzielenie na zbiór treningowy i testowy

In [57]:
# Dzielimy na podstawie is_train i usuwamy ją
train_dataset = full_dataset_transformed[full_dataset_transformed['is_train'] == 1].drop(columns=['is_train'])
test_dataset = full_dataset_transformed[full_dataset_transformed['is_train'] == 0].drop(columns=['is_train'])

In [58]:
record_number_train = len(train_dataset)
record_number_test = len(test_dataset)

print(f"Liczba rekordów w grupie treningowej: {record_number_train}")
print(f"Liczba rekordów w grupie testowej: {record_number_test}")

Liczba rekordów w grupie treningowej: 135397
Liczba rekordów w grupie testowej: 72907


### Sprawdzenie, czy gotowe do ML

In [59]:
# Sprawdzenie typów danych
print(train_dataset.dtypes)

# Sprawdzenie wartości brakujących
print(train_dataset.isnull().sum())

ID               float64
Cena             float64
Stan             float64
Rok_produkcji    float64
Przebieg_km      float64
                  ...   
Kolor_red          uint8
Kolor_silver       uint8
Kolor_violet       uint8
Kolor_white        uint8
Kolor_yellow       uint8
Length: 1352, dtype: object
ID               0
Cena             0
Stan             0
Rok_produkcji    0
Przebieg_km      0
                ..
Kolor_red        0
Kolor_silver     0
Kolor_violet     0
Kolor_white      0
Kolor_yellow     0
Length: 1352, dtype: int64


In [60]:
# Sprawdzenie typów danych
print(test_dataset.dtypes)

# Sprawdzenie wartości brakujących
print(test_dataset.isnull().sum())

ID               float64
Cena             float64
Stan             float64
Rok_produkcji    float64
Przebieg_km      float64
                  ...   
Kolor_red          uint8
Kolor_silver       uint8
Kolor_violet       uint8
Kolor_white        uint8
Kolor_yellow       uint8
Length: 1352, dtype: object
ID                   0
Cena             72907
Stan                 0
Rok_produkcji        0
Przebieg_km          0
                 ...  
Kolor_red            0
Kolor_silver         0
Kolor_violet         0
Kolor_white          0
Kolor_yellow         0
Length: 1352, dtype: int64


In [61]:
# Usuwanie kolumny Cena z test_dataset, która jest pusta
test_dataset = test_dataset.drop(columns=['Cena'])

#### Nie ma object, czyli dane są gotowe do użycia

## Zapis przygotowanych danych do csv

In [62]:
train_dataset.to_csv('train_dataset.csv', index=False)
test_dataset.to_csv('test_dataset.csv', index=False)